## Deploying one module on IoT Edge Device

In this notebook, we deploy a single module -- people-detection-service on IoT Edge. We want to make sure one-module deployment works before moving to two-module deployment.

In [9]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

import time
from dotenv import set_key, get_key, find_dotenv

In [10]:
# get the .env file where all the variables are stored
env_path = find_dotenv(raise_error_if_not_found=True)

In [11]:
resource_group = get_key(env_path, 'resource_group')
iot_hub_name = get_key(env_path, 'iot_hub_name')
device_id = get_key(env_path, 'device_id')


acr_name = get_key(env_path, 'acr_name')
acr_login_server = get_key(env_path, 'acr_login_server')
acr_password = get_key(env_path, 'acr_password')
mlimg_location = get_key(env_path, 'ml_img_location')

In [12]:
# deploy ML module
ml_module_name = 'people-detection-service'

In [13]:
# Based on the deployment_template.json file, we insert necessary information.

file = open('./deployment_peopledetection_template.json')
contents = file.read()

contents = contents.replace('__REGISTRY_SERVER_NAME', acr_login_server)
contents = contents.replace('__REGISTRY_PASSWORD', acr_password)
contents = contents.replace('__REGISTRY_USER_NAME', acr_name)
contents = contents.replace('__ML_MODULE_NAME', ml_module_name)
contents = contents.replace('__REGISTRY_ML_IMAGE_LOCATION', mlimg_location)

with open('./deployment.json', 'wt', encoding='utf-8') as output_file:
    output_file.write(contents)

In [14]:
print("start deployment process...")

start deployment process...


In [15]:
# Push the deployment JSON to the IOT Hub
!az iot edge set-modules --device-id $device_id \
                         --hub-name $iot_hub_name \
                         --content deployment.json \
                         -g $resource_group

[
  {
    "authentication": {
      "symmetricKey": {
        "primaryKey": "iX+Z5++8iO0xhBy55Ntqrmyk6gZA17fYLI2dS72wSkE=",
        "secondaryKey": "njdrCj367T1FZLedL5q/yQKXuxC1rXbs+5as5b4OWbw="
      },
      "type": "sas",
      "x509Thumbprint": {
        "primaryThumbprint": null,
        "secondaryThumbprint": null
      }
    },
    "cloudToDeviceMessageCount": 0,
    "connectionState": "Connected",
    "connectionStateUpdatedTime": "2019-08-14T18:57:22.414278",
    "deviceId": "gpudevice",
    "etag": "NDYyMjQxNDA3",
    "generationId": "637012891217367561",
    "lastActivityTime": "0001-01-01T00:00:00",
    "managedBy": null,
    "moduleId": "$edgeAgent"
  },
  {
    "authentication": {
      "symmetricKey": {
        "primaryKey": "WMtrfEPiSQDViOh8VuV/UH8jbAPyNwlqk1KzCTkwnpQ=",
        "secondaryKey": "B76PkpeGlL86cNBN50//5wYARhcimChjKCDOFKJwklo="
      },
      "type": "sas",
      "x509Thumbprint": {
        "primaryThumbprint": null,
        "secondaryThumbprint": null
    

In [16]:
# restart the daemon
!sudo systemctl restart iotedge
time.sleep(20) # Wait 20 seconds for iotedge to restart
# restart the daemon again
!sudo systemctl restart iotedge

When you run `docker ps` command in the edge device, you should see there are three containers running: `edgeAgent`, `edgeHub`, and two other containers with name `ml_module_name`.

In [ ]:
!docker ps

When the container `ml_module_name` is running, you can also test this module. We show how to do it at the notebook [05_TestPeopleDetectionModule.ipynb](./05_TestPeopleDetectionModule.ipynb).